In [1]:
from crewai import Crew, Task, Agent
from langchain_openai import ChatOpenAI
import os
from personal_search.tools.books import LibraryTool, YoutubeTool
import warnings
from pydantic import BaseModel, Field
from typing import List
from IPython.display import display, Markdown, Latex

warnings.filterwarnings("ignore")

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

# ------ Ollama - Local---------#
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
embeddings_model = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS_MODEL"]=embeddings_model # documentar como definir variáveis de ambiente

# ------ OpenAI - API ---------#
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# ------ DEBUG ---------#
os.environ["DEBUG"]="True"
debug = os.getenv("DEBUG_MODE", "False")
if debug.lower() == "true":
    VERBOSE = 2
else:
    VERBOSE = False

class TopicoEmLivros(BaseModel):
	"""Modelo de Tópico em Livros"""
	titulo: str = Field(..., description="Título do tópico")
	encontrado: bool = Field(..., description="O tópico foi encontrado em livros?")
	livros: List[str] = Field(..., description="Livros que tratam do tópico")
	citacoes: List[str] = Field(..., description="Citações de livros sobre o tópico")
	tratamento: str = Field(..., description="Como o tópico é tratado nos livros")
	fontes: List[str] = Field(..., description="Fontes do tópico")

In [2]:
# abrir arquivo yaml como dict
def load_yaml(file_path):
    import yaml
    with open(file_path, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

agents_config = load_yaml("src/personal_search/config/agents.yaml")
tasks_config = load_yaml("src/personal_search/config/tasks.yaml")

In [3]:
researcher = Agent(
	config=agents_config['book_researcher'],
    tools=[LibraryTool()],
    verbose=VERBOSE,
    llm=llm
)

research = Task(
	config=tasks_config['topic_in_books'],
    agent=researcher,
    output_pydantic=TopicoEmLivros,
)

crew = Crew(
  agents=[researcher],
  tasks=[research],
  verbose=2,
  memory=False,
  llm=llm,
  full_output=True,
)

input_text  = "O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas."

inputs = {
    "topic": input_text
}
result = crew.kickoff(inputs=inputs)

 [2024-07-24 11:02:00][DEBUG]: == Working Agent: Pesquisador de Livros

 [2024-07-24 11:02:00][INFO]: == Starting Task: Tópico: O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo. Então, quando eu for vê-los novamente, ou mesmo quando ouvir a seu respeito, saberei que estão firmes e unidos em um só espírito e em um só propósito, lutando juntos pela fé que é proclamada nas boas-novas.
Realize uma pesquisa em livros passando o tópico na íntegra, buscando responder os seguintes aspectos: - **Presença em livros:** O tópico é mencionado em algum livro relevante? Se sim, quais são esses livros e como eles abordam o tópico? - **Citação do trecho:** Forneça a citação do livro na íntegra retornada pela pesquisa de livros. Não elabore. - **Tratamento literário:** Como o tópico é tratado nos livros encontrados? Descreva os principais temas, abordagens e interpretações literárias. - **Fontes:** Cite todas as fontes utilizadas para a pesquisa, incluindo 

2024-07-24 11:02:04,257 - 134596476048448 - add_config.py-add_config:30 - WARNING: min_chunk_size 0 should be greater than chunk_overlap 100, otherwise it is redundant.


 

O Peregrino - John Bunyan - Página 138:
o som com olhares e gestos, o quanto Cristão e seu irmão eram bem-vindos, tamanho era o júbilo com que os acolhiam. E agora os dois homens, por assim dizer, se viam no céu antes de lá entrar, extasiados que estavam pela visão dos anjos e pela audição de notas tão melodiosas. Dali também já avistavam a própria cidade e julgaram ouvir todos os sinos repicando como sinal de boas-vindas. Mas, acima de tudo, estava a expectativa alegre e doce que tinham de lá viver ao lado de tão bela companhia, e isso pelos séculos dos séculos. Ah! Com que língua ou pena se pode expressar júbilo tão glorioso? E assim chegaram ao portão.

O Peregrino - John Bunyan - Página 96:
E SPERANÇOSO — Sem dúvida nenhuma você disse a verdade. Mas grande é a misericórdia que nos cobre, porque nem você nem eu nos transformamos em exemplo disso, principalmente eu. Isso nos dá oportunidade de agradecer a Deus, de temê-lo e de jamais esquecer a mulher de Ló.

Por que tarda o pleno

In [8]:

def print_markdown(result):
    markdown = ''
    keys = result.pydantic.dict().keys()
    result_dict = result.pydantic.dict()
    for key in keys:
        markdown += f"## {key.capitalize()}\n"
        # verifica se o valor é uma lista
        if isinstance(result_dict[key], list):
            for item in result_dict[key]:
                markdown += f"- {item}\n"
        else:
            markdown += f"{result_dict[key]}\n\n"
        markdown += "---\n\n"
    return markdown

markdown = print_markdown(result)
Markdown(markdown)

## Titulo
O mais importante é que vocês vivam em sua comunidade de maneira digna das boas-novas de Cristo

---

## Encontrado
True

---

## Livros
- O Peregrino
- Por que tarda o pleno Avivamento
---

## Citacoes
- "O júbilo e a expectativa dos personagens ao vislumbrarem o céu e a cidade celestial transmitem a alegria e a esperança da vida eterna ao lado de Deus." - O Peregrino, p. 138
- "Importância da misericórdia divina e da gratidão por ela" - O Peregrino, p. 96
- "Reflexão sobre a importância de agradar a Deus acima de tudo e a necessidade de uma fé viva e espiritual para comunicar a Palavra com poder" - Por que tarda o pleno Avivamento, p. 50
---

## Tratamento
Os autores exploram a jornada espiritual, a gratidão pela misericórdia divina e a necessidade de uma fé genuína e viva para viver de acordo com os princípios cristãos.

---

## Fontes
- O Peregrino - John Bunyan, 2010
- Por que tarda o pleno Avivamento - Leonard Ravenhill, 2008
---

